In [20]:
import pandas as pd
import numpy as nd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import joblib

In [21]:
file_path = "lap.csv"  
data = pd.read_csv(file_path)
# Assuming your dataset is in a CSV file named 'laptops_data.csv'
print(data['Brand name of your laptop?'])
data.drop("Timestamp",axis = 1,inplace = True)
data.drop("Email Address",axis = 1,inplace = True)
print(data['Brand name of your laptop?'].unique())

0       DELL
1     LENOVO
2     LENOVO
3       DELL
4         HP
       ...  
66      DELL
67      DELL
68    Others
69    Others
70        HP
Name: Brand name of your laptop?, Length: 71, dtype: object
['DELL' 'LENOVO' 'HP' 'ACER' 'ASUS' 'APPLE' 'Others']


In [22]:
data['RAM size of your current laptop'] = data['RAM size of your current laptop'].str.replace('[a-zA-Z]', '', regex=True)
data['Storage (in GB)'] = data['Storage (in GB)'].str.replace('[a-zA-Z]', '', regex=True)
data['Clock speed'] = data['Clock speed'].str.replace('[^a-zA-Z0-9]', '', regex=True)
data['weight(kgs)'] = data['weight(kgs)'].str.replace('[^a-zA-Z0-9]', '', regex=True)
data['Avg. Battery draining time(hours)'] = data['Avg. Battery draining time(hours)'].str.replace('^a-zA-Z0-9]', '', regex=True)

In [23]:
label_encoder = LabelEncoder()
data['Brand name of your laptop?'] = label_encoder.fit_transform(data['Brand name of your laptop?'])
data['Storage type'] = label_encoder.fit_transform(data['Storage type'])
data['Processor'] = label_encoder.fit_transform(data['Processor'])
data['Dedicated GPU'] = label_encoder.fit_transform(data['Dedicated GPU'])
data['Ethernet port'] = label_encoder.fit_transform(data['Ethernet port'])
data['Storage (in GB)'] = label_encoder.fit_transform(data['Storage (in GB)'])
data['Clock speed'] = label_encoder.fit_transform(data['Clock speed'])
data['Avg. Battery draining time(hours)'] = label_encoder.fit_transform(data['Avg. Battery draining time(hours)'])
data['weight(kgs)'] = label_encoder.fit_transform(data['weight(kgs)'])
data['screen size'] = label_encoder.fit_transform(data['screen size'])
data['RAM size of your current laptop'] = label_encoder.fit_transform(data['RAM size of your current laptop'])
# Features available for recommendation
available_features = [
    'Brand name of your laptop?',
    'RAM size of your current laptop',
    'Storage (in GB)',
    'Storage type',
    'Processor',
    'Clock speed',
    'Dedicated GPU',
    'weight(kgs)',
    'screen size',
    'Ethernet port',
    'Avg. Battery draining time(hours)'
]

In [24]:
# Split the data into features (X) and target variables (y)
X = data.drop(['Avg. Battery draining time(hours)'], axis=1)
y = data['Avg. Battery draining time(hours)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)

# Initialize the linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Save the trained model to a file
joblib.dump(model, 'laptop_recommendation_model.joblib')


    Brand name of your laptop?  RAM size of your current laptop  Storage type  \
30                           3                                0             1   
59                           3                                3             2   
35                           4                                0             1   
58                           6                                2             0   
42                           1                                3             1   
16                           1                                0             1   
34                           5                                3             1   
44                           5                                0             0   
28                           3                                0             1   
7                            5                                3             1   
54                           4                                2             2   
50                          

['laptop_recommendation_model.joblib']

In [27]:
def recommend_laptop(user_preferences):
    # Load the trained model
    loaded_model = joblib.load('laptop_recommendation_model.joblib')
    
    # Encode categorical variables
    user_preferences['Brand name of your laptop?'] = label_encoder.transform([user_preferences['Brand name of your laptop?']])[0]
    user_preferences['RAM size of your current laptop'] = label_encoder.transform([user_preferences['RAM size of your current laptop']])[0]
    user_preferences['Storage (in GB)'] = label_encoder.transform([user_preferences['Storage (in GB)']])[0]
    user_preferences['Storage type'] = label_encoder.transform([user_preferences['Storage type']])[0]
    user_preferences['Processor'] = label_encoder.transform([user_preferences['Processor']])[0]
    user_preferences['Clock speed'] = label_encoder.transform([user_preferences['Clock speed']])[0]
    user_preferences['Dedicated GPU'] = label_encoder.transform([user_preferences['Dedicated GPU']])[0]
    user_preferences['weight(kgs)'] = label_encoder.transform([user_preferences['weight(kgs)']])[0]
    user_preferences['screen size'] = label_encoder.transform([user_preferences['screen size']])[0]
    user_preferences['Ethernet port'] = label_encoder.transform([user_preferences['Ethernet port']])[0]
    user_preferences['Avg. Battery draining time(hours)'] = label_encoder.transform([user_preferences['Avg. Battery draining time(hours)']])[0]
    # Make prediction
    predicted_battery_drain = loaded_model.predict([user_preferences])[0]
    
    # Filter laptops based on user preferences
    filtered_laptops = data.copy()
    for feature in available_features:
        if feature not in user_preferences:
            filtered_laptops = filtered_laptops[filtered_laptops[feature] == user_preferences[feature]]

    # Sort laptops by predicted Battery Draining Time
    filtered_laptops['Battery Predicted'] = abs(filtered_laptops['Avg. Battery draining time(hours)'] - predicted_battery_drain)

    # Calculate a weighted score based on the differences
    filtered_laptops['Score'] = filtered_laptops['Battery Predicted']

    # Sort by score and return the top recommendation
    top_recommendation = filtered_laptops.sort_values(by='Score').head(1)

    return top_recommendation[available_features]


In [28]:
print("Welcome to the Laptop Recommendation System!")
print("Please provide your preferences:")

user_preferences = {}
for feature in available_features:
    if feature != 'Avg. Battery draining time(hours)':  # Exclude the target feature from user input
        user_input = input(f"Enter your preference for {feature}: ")
        user_preferences[feature] = user_input

print(user_preferences)

recommendation = recommend_laptop(user_preferences)
print("\nTop Recommended Laptop:")
print(recommendation)


Welcome to the Laptop Recommendation System!
Please provide your preferences:
